In [1]:
import pandas as pd
import json

In [2]:
ls

GNO_v2.ipynb


In [3]:
cncr_typs=['ACC','BLCA','BRCA','CESC','COADREAD',
            'ESCC','GEA','HNSC','KIRCKICH','KIRP',
            'LGGGBM','LIHCCHOL','LUAD','LUSC','MESO',
            'OV','PAAD','PCPG','PRAD','SARC',
            'SKCM','TGCT','THCA','THYM','UCEC',
            'UVM']

In [4]:
ls

2020-04-08.zip        2020-04-08_extracted/ GNO_v2.ipynb


In [5]:
cd 2020-04-08_extracted/

/Users/karlberb/work/tmp/v8/Results/GNO_v2/2020-04-08_extracted


In [6]:
ls

ACC_features_sets.tsv                   LUSC_features_sets.tsv
ACC_single_sample_predictions.tsv       LUSC_single_sample_predictions.tsv
BLCA_features_sets.tsv                  MESO_features_sets.tsv
BLCA_single_sample_predictions.tsv      MESO_single_sample_predictions.tsv
BRCA_features_sets.tsv                  OV_features_sets.tsv
BRCA_single_sample_predictions.tsv      OV_single_sample_predictions.tsv
CESC_features_sets.tsv                  PAAD_features_sets.tsv
CESC_single_sample_predictions.tsv      PAAD_single_sample_predictions.tsv
COADREAD_features_sets.tsv              PCPG_features_sets.tsv
COADREAD_single_sample_predictions.tsv  PCPG_single_sample_predictions.tsv
ESCC_features_sets.tsv                  PRAD_features_sets.tsv
ESCC_single_sample_predictions.tsv      PRAD_single_sample_predictions.tsv
GEA_features_sets.tsv                   SARC_features_sets.tsv
GEA_single_sample_predictions.tsv       SARC_single_sample_predictions.tsv
HNSC_features_sets.tsv                

In [76]:
count=0
header = None
row_range=list(range(5,2094))
with open("ACC_single_sample_predictions.tsv", "rt", encoding='utf-8') as handle:
    for line in handle:
        count=count+1
        if count > 4360:
            break
        if not line.startswith("#"):
            row = line.rstrip().split("\t") # row object is a list
            if header is None:
                header = row # first time through loop
            else:
                sample = row[0]
                repeat = row[1]
                fold = row[2]
                test = row[3]
                label = row[4]
                for i in row_range:
                    meta = json.loads(row[i].replace("'", "\""))
                    cls = pd.Series(meta['classification'])
                    c = cls.idxmax()
                    #print("%s == %s" % (label, c))

KeyboardInterrupt: 

In [75]:
header

['',
 'Repeat',
 'Fold',
 'Test',
 'Label',
 'conf0|6|2020-03-13 08:17:44.059|p',
 'conf1|1|2020-03-13 08:17:44.059|p',
 'conf2|5|2020-03-13 08:17:44.059|p',
 'conf3|9|2020-03-13 08:17:44.059|p',
 'conf4|1|2020-03-13 08:17:44.059|p',
 'conf5|4|2020-03-13 08:17:44.059|p',
 'conf6|1|2020-03-13 08:17:44.059|p',
 'conf7|7|2020-03-13 08:17:44.059|p',
 'conf8|1|2020-03-13 08:17:44.059|p',
 'conf9|8|2020-03-13 08:17:44.059|p',
 'conf10|6|2020-03-13 08:17:44.059|p',
 'conf11|2|2020-03-13 08:17:44.059|p',
 'conf12|4|2020-03-13 08:17:44.059|p',
 'conf13|5|2020-03-13 08:17:44.059|p',
 'conf14|7|2020-03-13 08:17:44.059|p',
 'conf15|9|2020-03-13 08:17:44.059|p',
 'conf16|1|2020-03-13 08:17:44.059|p',
 'conf17|1|2020-03-13 08:17:44.059|p',
 'conf18|6|2020-03-13 08:17:44.059|p',
 'conf19|6|2020-03-13 08:17:44.059|p',
 'conf20|7|2020-03-13 08:17:44.059|p',
 'conf21|2|2020-03-13 08:17:44.059|p',
 'conf22|8|2020-03-13 08:17:44.059|p',
 'conf23|5|2020-03-13 08:17:44.059|p',
 'conf24|2|2020-03-13 08:17:44

In [72]:
len(header)

2094

In [42]:
label

'ACC:2'

In [47]:
meta2

{'range': {'min': 0, 'max': 1},
 'classification': {'ACC:1': 0.76282, 'ACC:2': 0.21537, 'ACC:3': 0.02181}}

In [53]:
row_range=list(range(5,2094))

In [74]:
len(row_range)

2089

In [70]:
row[2000]

'{"range":{ "min":0,"max":1}, "classification":{"ACC:1":0.08323,"ACC:2":0.59853,"ACC:3":0.31824}}'

In [10]:
gnosis_scores=pd.DataFrame(columns=['Label','Score','G_mod','C_typ']) # main storage
#cncr_typs=['ACC']
for c_typ in cncr_typs:
    print(c_typ)     # Read Gnosis Predictions
    gp=pd.read_csv('/Users/karlberb/work/tmp/v8/Results/GNO_v2/2020-04-08_extracted/'+c_typ+'_single_sample_predictions.tsv',delimiter='\t')
    gp=gp[(gp['Test']==1)].copy() # Filter prediction file for test set
    gp=gp.iloc[:, 4:]
    label=gp["Label"].str[-1]    # Store subtype labels
    
    for i in list(range(1,10)):         ### go through nine column chunks
        modeln='model'+str(i)
        print(modeln)
        gp2=gp.filter(regex=modeln)   # Collapse horizontally on modeln
        gp2.insert(0,'Lbl',label)
        score_list=[]
        label_list=[]
        df=pd.DataFrame(columns=['Label', 'Score'])
        max_s=gp2['Lbl'].max()
        plos=int(max_s)+1
        
        for j,Labl in enumerate(gp2.Lbl): # Count across each row
            var=gp2.iloc[j,:].str[-1]
            scores=var.value_counts(1)
            score=scores[Labl]
            score_list=score_list+[score]
            label_list=label_list+[Labl]
        d = {'Label': label_list, 'Score': score_list}
        df = pd.DataFrame(d)
        
        mean_labels=[]
        mean_scores=[]
        for k in list(range(1, plos)):
            Sub_Temp = df[df.Label.eq(str(k))]
            scr_mn=Sub_Temp["Score"].mean()
            mean_scores=mean_scores+[scr_mn]
            mean_labels=mean_labels+[k]
        d2 = {'Label': mean_labels, 'Score': mean_scores}    # Mean across convergences and samples per subtype
        df2 = pd.DataFrame(d2)
        
        df2['G_mod']=modeln
        df2['C_typ']=str(c_typ)
        gnosis_scores = gnosis_scores.append(df2, ignore_index=True)
    break

ACC


ParserError: Error tokenizing data. C error: Expected 1 fields in line 4179, saw 2094
